# Refactored Toolkit

The starting point is a session to a data server
(the name `DataServerSession` might change)

In [18]:
from zhinst.toolkit import DataServerSession

session =  DataServerSession()

It es an aquivalent to the daq object in ziPython and offers the same functionality
(module, connecting devices, ...).

In a addition it implements the "ZI" common nodes as a lazy nodetree

(if necessary one can easily acces the underlying daq object with session.daq_server)

In [19]:
# Accessing zi nodes
session.about.version()

'22.01'

In [20]:
#accessing modules
session.awg_module

Modules accessed through toolkit are the same as the would be in ziPython, with the small but handy addition of having
a lazy nodetree injected.
(session.create_***_module() can be used to create a second instance of a module if necessary)

In [21]:
session.awg_module.nodetree.compiler.status()

-1

### Connect a device

Like in ziPython one can connect a device to a data server. If the device is not connected one can access it
directly without the need of connecting it, although connecting an already connected device does not harm (the
same way it doesn`t in ziPython)

In [22]:
device = session.connect_device("dev12033")
device

SHFQA(SHFQA4,dev12033)

In [23]:
session.devices["dev12033"] == device

True

## Lazy Nodetree

All nodes from the device are automatically accessible via the lazy nodetree

Since the nodetree is generated lazy the setup time is close to 0

In [24]:
dir(device)

['clockbase',
 'dios',
 'features',
 'max_qubits_per_channel',
 'num_qachannels',
 'num_scopes',
 'qachannels',
 'scopes',
 'stats',
 'status',
 'system']

In [25]:
print(device.qachannels[0].oscs[0].freq.node)
print(device.qachannels[0].oscs[0].freq.description)
print(device.qachannels[0].oscs[0].freq.type)
print(device.qachannels[0].oscs[0].freq.unit)
print(device.qachannels[0].oscs[0].freq.options)
print(device.qachannels[0].oscs[0].freq.properties)

/DEV12033/QACHANNELS/0/OSCS/0/FREQ
Controls the frequency of each digital Oscillator.
Double
Hz
{}
Read, Write, Setting


Since the nodetree is lazy the nodetree only checks if the node is valid once 
one performs operations on it

In [26]:
device.weird.name.not_.existing

/dev12033/weird/name/not/existing

In [27]:
device.weird.name.not_.existing.unit

KeyError: '/dev12033/weird/name/not/existing'

To get or set values one can use the call function. No argument gets the value
and a single argument sets the provided value.

(the nodetree automatically matches the call to the right ziPython function)

the following kwargs are provided

        deep (bool): Flag if the get operation should return the cached value from
            the dataserver or get the value from the device, which is significantly
            slower.

In [28]:
device.qachannels[0].oscs[0].freq(deep=True)

(102911128987440, 100000000.00000142)

In [29]:
value = device.qachannels[0].oscs[0].freq()
print(value)
device.qachannels[0].oscs[0].freq(value + 1e6, deep=True)
print(device.qachannels[0].oscs[0].freq())
device.qachannels[0].oscs[0].freq(value)

100000000.00000142
100999999.99999909


### Wildcards

The nodetree fully supports wildcards. This means getting, setting, subscribing, waiting
for wildcard nodes is possible.
(all unix wildcards are supported but * is probably the most usefull)

In [30]:
device.qachannels[0]["*/*"].freq()

{/DEV12033/QACHANNELS/0/OSCS/0/FREQ: (102925809018788, 100000000.00000142)}

In [31]:
device.qachannels["*"].oscs["*"].freq()

{/DEV12033/QACHANNELS/0/OSCS/0/FREQ: (102929288991224, 100000000.00000142),
 /DEV12033/QACHANNELS/1/OSCS/0/FREQ: (102929288991224, 100000000.00000142),
 /DEV12033/QACHANNELS/2/OSCS/0/FREQ: (102929288991224, 100000000.00000142),
 /DEV12033/QACHANNELS/3/OSCS/0/FREQ: (102929288991224, 100000000.00000142)}

In [32]:
device.qachannels["*"].oscs["*"].freq(100e6)

In [33]:
device.qachannels["*"].oscs["*"].freq()

{/DEV12033/QACHANNELS/0/OSCS/0/FREQ: (102940129025612, 100000000.00000142),
 /DEV12033/QACHANNELS/1/OSCS/0/FREQ: (102940129025612, 100000000.00000142),
 /DEV12033/QACHANNELS/2/OSCS/0/FREQ: (102940129025612, 100000000.00000142),
 /DEV12033/QACHANNELS/3/OSCS/0/FREQ: (102940129025612, 100000000.00000142)}

In [34]:
result = device.qachannels["*"].oscs["*"].freq()
result[device.qachannels[0].oscs[0].freq]

(102944850043496, 100000000.00000142)

### Transactional set
wildcards use a transactional set and get in the under the hood. In addition one can directly
perform transactional sets with a `with` statement. Every set command inside this
statement will be cached and send as a single transaction to the device.

In [35]:
with device.set_transaction():
    device.qachannels[0].centerfreq(1e6)
    device.qachannels[0].mode(0)

# Polling

The refactored toolkit also adds support for polling. Since polling is done on a
data server level and not device specific it is implemented in the session class

In [36]:
device.dios[0].output.subscribe()
import time
device.dios[0].output(1)
time.sleep(0.5)
device.dios[0].output(0)
time.sleep(0.5)
device.dios[0].output(1)
time.sleep(0.5)
device.dios[0].output(0)
polled_data = session.poll()
polled_data

{SHFQA(SHFQA4,dev12033): {/DEV12033/DIOS/0/OUTPUT: {'timestamp': array([102954849027060, 102956850113172, 102958888988684, 102960929019096],
         dtype=uint64),
   'value': array([1, 0, 1, 0], dtype=int64)}}}

The polled data is a dictionary [node] 

In [37]:
polled_data[device.dios[0].output]

{'timestamp': array([102954849027060, 102956850113172, 102958888988684, 102960929019096],
       dtype=uint64),
 'value': array([1, 0, 1, 0], dtype=int64)}

# Additional functionalities
Besides the lazy nodetree and the functionalities from the session and BaseInstrument
toolkit provides additional functions for some instruments. 

Right now SHFQA and SHFSG do have such functions. The other devices join before
the release. 

The additional functions are embedded into the nodetree. Once the documentation
is up and running one can easily see the functionality there. For now the
functionality can be look up in the code directly or via the python help. 

Everything inside the old toolkit is more or less also available in the new one.

For the SHFQA the device utiliy functions are already embedded

In the following a few functions are shown:

In [38]:
help(device.qachannels[0].configure_channel)

Help on method configure_channel in module zhinst.toolkit.driver.shfqa:

configure_channel(input_range: int, output_range: int, center_frequency: float, mode: zhinst.toolkit.interface.SHFQAChannelMode) -> None method of zhinst.toolkit.driver.shfqa.QAChannel instance
    Configures the RF input and output of a specified channel.
    
    Args:
        input_range (int): maximal range of the signal input power in dbM
        output_range (int): maximal range of the signal output power in dbM
        center_frequency (float): center Frequency of the analysis band
        mode (SHFQAChannelMode): select between spectroscopy and readout mode.



In [39]:
help(device.qachannels[0].sweeper.run)

Help on method run in module zhinst.toolkit.driver.modules.shfqa_sweeper:

run() method of zhinst.toolkit.driver.modules.shfqa_sweeper.SHFSweeper instance
    Perform a sweep with the specified settings.
    
    This method eventually wraps around the `run` method of
    `zhinst.utils.shf_sweeper`



# Sequencer and Waveform

In contrast to the old toolkit the refactoring decouples the waveform and
sequences from the devices. 

What happens to the predefined sequences will be descided. For now one has to
define them by oneself.

In [41]:
sequencer_code="""\
wave w0_1 = placeholder(1008);
wave w0_2 = placeholder(1008);
wave w1_1 = placeholder(1008);
wave w1_2 = placeholder(1008);

assignWaveIndex(1,2,w0_1,1,2,w0_2,0);
assignWaveIndex(1,2,w1_1,1,2,w1_2,2);
"""

The waveforms are wrapped in a mutuable dictionary called `Waveforms`

each waveform consists of a tuple(wave1,wave2,markers).
(For more infos see help(Waveforms))

**INFO** For the SHFQA the class is called `SHFQAWaveforms`  

In [42]:
from zhinst.toolkit import Waveforms
import numpy as np

waveforms = Waveforms()
waveforms[0] = (np.ones(1008), -np.ones(1008))

In [43]:
device2 = session.connect_device("DEV12029")

device2.sgchannels[0].awg.load_sequencer_program(sequencer_code)
device2.sgchannels[0].awg.write_to_waveform_memory(waveforms)

# waveforms can also be read from the device
waveform_device = device2.sgchannels[0].awg.read_from_waveform_memory()
len(waveform_device)

2